In [ ]:
import os
import numpy as np
import csv
import yaml

: 

### LWC version - from csv file

In [ ]:
# Generate crypt_engine_stats.yml file for LWC finalists + other selected ciphers

"""
crypt_stats = []

# AES-GCM Pipelined
stat_aesgcm_pipelined = {'family': 'aesgcm', 
                         'name': 'pipeline', 
                         'auth-enc-parallel': True,
                         'enc-cycle-per-128bit': 1, 
                         'auth-cycle-per-128bit': 1, 
                         'auth-additional-cycle-per-block': 1,               # One AES for initializing per block
                         'xor-cycle': 1, 
                         'enc-energy-per-128bit': 1.29 * 128, 
                         'auth-energy-per-128bit': 57.7, 
                         'auth-additional-energy-per-block': 1.29 * 128,     # One AES for initializing per block
                         'xor-energy-per-128bit': 0}
"""

lwc_spec = '/home/ubuntu/squareloop/benchmarks/test_case/16_PE/crypto_spec/lwc_spec.csv'

crypt_stats = []

with open(lwc_spec, mode = 'r' , encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    for row in reader:
        temp = {'family': row['Name'], 
                'name': row['Tag'], 
                'auth-enc-parallel': (row['AuthEncParallel']  == '1'),
                'datapath': int(row['Datapath']),
                'enc-cycle-per-datapath': int(row['EncryptionCycle']),
                'auth-cycle-per-datapath': int(row['AuthenticationCycle']),
                'auth-additional-cycle-per-block': int(row['AuthenticationPerBlockCycle']), 
                'xor-cycle':0, 
                'enc-energy-per-datapath': 0,
                'auth-energy-per-datapath': 0, 
                'auth-additional-energy-per-block': 0, 
                'xor-energy-per-datapath': 0, 
                'hash-size': int(row['TagSize'])}

        crypt_stats.append(temp)

with open(os.path.join('/home/workspace', 'lwc_stats.yaml'), 'w') as f:
    _ = yaml.dump(crypt_stats, f)

### Old version

In [ ]:
# Define the crypt engine & baseline dram model
DRAM_NAME = 'lpddr4'
DRAM_ENERGY_PER_BIT = 8 # pj/bit

CRYPT_FAMILY = 'ascon'
CRYPT_NAME = '128a_1cycle'

CRYPT_AUTH_ENC_PARALLEL = False

CRYPT_ENC_CYCLE_PER_128BITS = 8
CRYPT_AUTH_CYCLE_PER_128BITS = 0
CRYPT_AUTH_ADDITIONAL_CYCLE_PER_BLOCK = (12 + 12) / 1 + 1 + 1 # initialization + finalization + 2 xors
CRYPT_XOR_CYCLE = 1

# ignore energy for now
CRYPT_ENC_ENERGY_PER_128BITS = 0
CRYPT_AUTH_ENERGY_PER_128BITS = 0
CRYPT_AUTH_ADDITIONAL_ENERGY_PER_BLOCK = 0
CRYPT_XOR_ENERGY = 0

In [ ]:
# Check the effective dram tables
effective_dram_tables_root = '/home/workspace/effective_dram_tables'
effective_model_name = '{}_{}_{}_data'.format(DRAM_NAME, CRYPT_FAMILY, CRYPT_NAME)
if not os.path.exists(os.path.join(effective_dram_tables_root, effective_model_name)):
    os.mkdir(os.path.join(effective_dram_tables_root, effective_model_name))
    os.mkdir(os.path.join(effective_dram_tables_root, effective_model_name, 'data'))

"""
name: lpddr4_aesgcm_1cycle_table
technology: effective_lpddr4_aesgcm_1cycle
accuracy: 100
path_to_data_dir: ./data # this is my own path
"""
yaml_dict = {'name': '{}_{}_{}_table'.format(DRAM_NAME, CRYPT_FAMILY, CRYPT_NAME), \
             'technology': 'effective_{}_{}_{}'.format(DRAM_NAME, CRYPT_FAMILY, CRYPT_NAME), \
             'accuracy': 100, \
             'path_to_data_dir': './data'}
yaml_file_name = '{}_{}_{}.table.yaml'.format(DRAM_NAME, CRYPT_FAMILY, CRYPT_NAME)
with open(os.path.join(effective_dram_tables_root, effective_model_name, yaml_file_name), 'w') as f:
    _ = yaml.dump(yaml_dict, f)

energy_per_128bit = DRAM_ENERGY_PER_BIT * 128 + (CRYPT_ENC_ENERGY_PER_128BITS + CRYPT_AUTH_ENERGY_PER_128BITS)

csv_dict_read = {'technology': 'effective_{}_{}_{}'.format(DRAM_NAME, CRYPT_FAMILY, CRYPT_NAME), \
                 'width': 128, \
                 'action': 'read', \
                 'energy': energy_per_128bit, \
                 'area': 0}
csv_dict_write = {'technology': 'effective_{}_{}_{}'.format(DRAM_NAME, CRYPT_FAMILY, CRYPT_NAME), \
                  'width': 128, \
                  'action': 'write', \
                  'energy': energy_per_128bit, \
                  'area': 0}
csv_dict_idle = {'technology': 'effective_{}_{}_{}'.format(DRAM_NAME, CRYPT_FAMILY, CRYPT_NAME), \
                 'width': 128, \
                 'action': 'read', \
                 'energy': 0, \
                 'area': 0}
with open(os.path.join(effective_dram_tables_root, effective_model_name, 'data/DRAM.csv'), 'w') as f:
    header = ['technology', 'width', 'action', 'energy', 'area']
    writer = csv.DictWriter(f, fieldnames=header)

    writer.writeheader()
    writer.writerow(csv_dict_read)
    writer.writerow(csv_dict_write)
    writer.writerow(csv_dict_idle)


In [ ]:
# Effective DRAM bandwidth
DRAM_BANDWIDTH = 32 # words / cycle, 1 word = 16 bits
NUM_ENGINES_PER_DATATYPE = 1
CRYPT_ENGINE_BANDWIDTH = NUM_ENGINES_PER_DATATYPE * (128 / 16) / CRYPT_ENC_CYCLE_PER_128BITS
EFFECTIVE_BANDWIDTH = min(DRAM_BANDWIDTH, CRYPT_ENGINE_BANDWIDTH)
print(EFFECTIVE_BANDWIDTH)

In [ ]:
# Generate yaml for crypt engine stats

crypt_stats = []

# AES-GCM Pipelined
stat_aesgcm_pipelined = {'family': 'aesgcm', 
                         'name': 'pipeline', 
                         'auth-enc-parallel': True,
                         'enc-cycle-per-128bit': 1, 
                         'auth-cycle-per-128bit': 1, 
                         'auth-additional-cycle-per-block': 1,               # One AES for initializing per block
                         'xor-cycle': 1, 
                         'enc-energy-per-128bit': 1.29 * 128, 
                         'auth-energy-per-128bit': 57.7, 
                         'auth-additional-energy-per-block': 1.29 * 128,     # One AES for initializing per block
                         'xor-energy-per-128bit': 0}

# AES-GCM Parallel
stat_aesgcm_parallel = {'family': 'aesgcm', 
                        'name': 'parallel', 
                        'auth-enc-parallel': True, 
                        'enc-cycle-per-128bit': 11, 
                        'auth-cycle-per-128bit': 8, 
                        'auth-additional-cycle-per-block': 11,               # One AES for initializing per block
                        'xor-cycle': 1, 
                        'enc-energy-per-128bit': 1.52 * 128, 
                        'auth-energy-per-128bit': 82.4, 
                        'auth-additional-energy-per-block': 1.52 * 128,     # One AES for initializing per block
                        'xor-energy-per-128bit': 0}

# AES-GCM Serial
stat_aesgcm_serial = {'family': 'aesgcm', 
                      'name': 'serial', 
                      'auth-enc-parallel': True, 
                      'enc-cycle-per-128bit': 336, 
                      'auth-cycle-per-128bit': 128, 
                      'auth-additional-cycle-per-block': 336,               # One AES for initializing per block
                      'xor-cycle': 1, 
                      'enc-energy-per-128bit': 6.0 * 128, 
                      'auth-energy-per-128bit': 345.6, 
                      'auth-additional-energy-per-block': 6.0 * 128,        # One AES for initializing per block
                      'xor-energy-per-128bit': 0}

# ASCON 128a 1cycle
stat_ascon_128a_1cycle = {'family': 'ascon', 
                          'name': '128a_1cycle', 
                          'auth-enc-parallel': False, 
                          'enc-cycle-per-128bit': 8, 
                          'auth-cycle-per-128bit': 0, 
                          'auth-additional-cycle-per-block': 26,     # 12 cycles for initialize/finalize, 2 cycles for XORs
                          'xor-cycle': 1, 
                          'enc-energy-per-128bit': 0, 
                          'auth-energy-per-128bit': 0, 
                          'auth-additional-energy-per-block': 0, 
                          'xor-energy-per-128bit': 0}
                          
# ASCON 128a 2cycle
stat_ascon_128a_2cycle = {'family': 'ascon', 
                          'name': '128a_2cycle', 
                          'auth-enc-parallel': False, 
                          'enc-cycle-per-128bit': 4, 
                          'auth-cycle-per-128bit': 0, 
                          'auth-additional-cycle-per-block': 14,     # 6 cycles for initialize/finalize, 2 cycles for XORs
                          'xor-cycle': 1, 
                          'enc-energy-per-128bit': 0, 
                          'auth-energy-per-128bit': 0, 
                          'auth-additional-energy-per-block': 0, 
                          'xor-energy-per-128bit': 0}


crypt_stats = [stat_aesgcm_pipelined, 
               stat_aesgcm_parallel, 
               stat_aesgcm_serial,
               stat_ascon_128a_1cycle, 
               stat_ascon_128a_2cycle]

with open(os.path.join('/home/workspace', 'crypt_engine_stats.yaml'), 'w') as f:
    _ = yaml.dump(crypt_stats, f)
